In [11]:
import os
import pandas as pd
from stt_utils import WORKING_DIR, create_dir_if_not_exists, get_logger
# importing the necessary modules:
from wordcloud import STOPWORDS 
import json
import re
from collections import Counter

In [12]:
unwanted_words_in_wordcloud = ["will", "okay"]

for w in unwanted_words_in_wordcloud:
    STOPWORDS.add(w)
stopwords = set(STOPWORDS)

WORDCLOUD_OUTPUT_DIR = os.path.join(WORKING_DIR, "wordcloud")   # the dir where script should save results
create_dir_if_not_exists(WORDCLOUD_OUTPUT_DIR)  # calling this function will create dir if it does not exists
TRANSCRIPTIONS_DIR = os.path.join(WORKING_DIR, "transcriptions")    # this is input dir, from where script find the json files
all_transcripts = [file for file in os.listdir(TRANSCRIPTIONS_DIR) if file.endswith(".json")]  # get all json files 


In [14]:
def read_transcription(json_path):
    with open(json_path, "r") as fp:
        json_data = json.load(fp)
    try:
        text = json_data["text"]
    except KeyError:
        print(f"text not present in {json_file_name}, skipping this file..")
        text = ""
    return text

def count_words(all_text):
    all_text = all_text.lower()
    all_text = re.sub('[^A-Za-z ]+', '', all_text)
    all_text = [word for word in all_text.split() if word not in stopwords]
    df = pd.DataFrame()
    df["word"] = dict(Counter(all_text)).keys()
    df["count"] = dict(Counter(all_text)).values()
    return df.sort_values(by="count", ascending=False).reset_index(drop=True)


# Generate Overview

all_text = ""
print(f"Creating wordcloud overview..")
for json_file_name in all_transcripts:
    json_filepath = os.path.join(TRANSCRIPTIONS_DIR, json_file_name)
    all_text += read_transcription(json_filepath).strip()
    
df = count_words(all_text)
df.to_csv(os.path.join(WORDCLOUD_OUTPUT_DIR, "wordcloud_overview.csv"))
print(f"Created wordcloud overview for and saved in wordcloud folder")

Creating wordcloud overview..
Created wordcloud overview for and saved in wordcloud folder
